In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("AI_Engineer_Book.pdf")
docs = loader.load()

In [2]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

In [3]:
chunks = text_splitter.split_documents(docs)

In [7]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = OllamaEmbeddings(
    model="nomic-embed-text"
)

In [8]:
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini")

In [28]:
from langchain_core.prompts import PromptTemplate

hyde_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
    Write a detailed, technical answer to the following question.
    This answer will be used for information retrieval.

    Question: {question}
    """
)

In [29]:
def generate_hypothetical_doc(question):
    prompt_text = hyde_prompt.format(question=question)
    response = llm.invoke(prompt_text)

    # Safety: ensure string
    if hasattr(response, "content"):
        return response.content
    return response

In [30]:
def hyde_retrieval(question, k=4):
    hypothetical_doc = generate_hypothetical_doc(question)
    
    docs = vectorstore.similarity_search(
        hypothetical_doc,
        k=k
    )
    return docs

In [31]:
def answer_with_rag(question):
    retrieved_docs = hyde_retrieval(question)

    context = "\n\n".join(
        doc.page_content for doc in retrieved_docs
    )

    final_prompt = f"""
    Answer the question using ONLY the context below.

    Context:
    {context}

    Question:
    {question}
    """

    return llm.invoke(final_prompt)

In [33]:
question = "What is Context Engineering?"
response = answer_with_rag(question)
print(response)

content='The provided context does not contain any information about Context Engineering.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 200, 'total_tokens': 212, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'cost': 3.72e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': 3.72e-05, 'upstream_inference_prompt_cost': 3e-05, 'upstream_inference_completions_cost': 7.2e-06}}, 'model_provider': 'openai', 'model_name': 'openai/gpt-4o-mini', 'system_fingerprint': 'fp_1590f93f9d', 'id': 'gen-1769797307-7RRgnMc8ZdTHOy3oAg0f', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c1023-cbb1-7b82-9e09-c66d06d43195-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 200, 'output_tokens': 12, 'total_tokens': 212, 'input_token_details